In [45]:
import pandas as pd

df = pd.read_csv("../outputdata/results.csv")

df.drop_duplicates(subset=None, keep='first', inplace=False)

df = df.drop(['source_case','similar_case','source_case_topic','similarity_score'], axis='columns')

df = df[df["citation_link"] == True]

df = df.groupby(['method','citation_link']).citation_link.agg('count').to_frame('counts').reset_index()
df = df.sort_values('counts',ascending=False)

df.head(20)

#df.groupby(['method','citation_link']).count().sort_values('citation_link',ascending=False)

,method,citation_link,counts
22,tfidf,True,354
4,doc2vec_256,True,35
5,doc2vec_256_10,True,34
0,doc2vec_128,True,34
1,doc2vec_128_10,True,33
11,doc2vec_256_e10,True,30
19,doc2vec_64_10,True,29
12,doc2vec_300_15,True,28
21,doc2vec_64_e10,True,27
7,doc2vec_256_15,True,27


In [17]:
tfidf_df = df[df["method"].str.contains("tfidf", case=False)]
tfidf_df.count

<bound method DataFrame.count of       source_case     similar_case  similarity_score method  citation_link  \
0     62003CJ0453      62006CO0421          0.727373  tfidf           True   
1     62003CJ0453      61984CJ0028          0.664869  tfidf          False   
2     62003CJ0453      61984CJ0195          0.618935  tfidf          False   
3     62003CJ0453      61990CJ0039          0.579735  tfidf          False   
4     62003CJ0453      62002CJ0145          0.566752  tfidf          False   
5     62003CJ0453      61987CJ0029          0.492766  tfidf          False   
6     62003CJ0453      61985CJ0363          0.489782  tfidf          False   
7     62003CJ0453      61985CJ0361          0.381843  tfidf          False   
8     62003CJ0453      62002CJ0286          0.379327  tfidf          False   
9     62003CJ0453      61982CJ0045          0.368112  tfidf          False   
10    62003CJ0453      61980CJ0066          0.334393  tfidf          False   
11    62003CJ0453      61994CJ0

In [18]:
doc2vec_only_df = df[df["method"].str.contains("doc2vec", case=False)]

doc2vec_only_df.count

<bound method DataFrame.count of        source_case similar_case  similarity_score              method  \
3540   62003CJ0453  62007CJ0520          0.723833          doc2vec_64   
3541   62003CJ0453  62006CJ0275          0.691029          doc2vec_64   
3542   62003CJ0453  62008CO0159          0.685423          doc2vec_64   
3543   62003CJ0453  62010CJ0071          0.683560          doc2vec_64   
3544   62003CJ0453  62008CJ0172          0.670302          doc2vec_64   
3545   62003CJ0453  62010CJ0163          0.656506          doc2vec_64   
3546   62003CJ0453  62010CJ0411          0.651202          doc2vec_64   
3547   62003CJ0453  62007CO0557          0.649508          doc2vec_64   
3548   62003CJ0453  62003CJ0499          0.648127          doc2vec_64   
3549   62003CJ0453  62008CJ0583          0.632056          doc2vec_64   
3550   62003CJ0453  62010CJ0454          0.625515          doc2vec_64   
3551   62003CJ0453  62008CJ0428          0.622548          doc2vec_64   
3552   62003CJ0453

In [23]:
doc2vec_ten_epochs_df = doc2vec_only_df[doc2vec_only_df["method"].str.contains("e10", case=False)]
doc2vec_twenty_epochs_df = doc2vec_only_df[~doc2vec_only_df["method"].str.contains("e10", case=False)]

doc2vec_twenty_epochs_df_true = doc2vec_twenty_epochs_df[doc2vec_twenty_epochs_df["citation_link"] == True]
doc2vec_ten_epochs_df_true = doc2vec_ten_epochs_df[doc2vec_ten_epochs_df["citation_link"] == True]

print(doc2vec_twenty_epochs_df_true.count)
print()
print()
print("**************************************")
print()
print()
print(doc2vec_ten_epochs_df_true.count)

<bound method DataFrame.count of        source_case similar_case  similarity_score          method  \
4087   62013CJ0011  62010CJ0420          0.663567      doc2vec_64   
4245   62001CJ0135  62001CJ0362          0.714962      doc2vec_64   
5342   62013CJ0011  62010CJ0420          0.635790     doc2vec_128   
5507   62001CJ0135  62001CJ0362          0.623458     doc2vec_128   
6289   62013CJ0528  62013CJ0307          0.478809     doc2vec_256   
6602   62013CJ0011  62010CJ0420          0.615066     doc2vec_256   
6774   62001CJ0135  62001CJ0362          0.571854     doc2vec_256   
7407   62003CJ0211  62007CJ0140          0.669704   doc2vec_64_10   
7862   62013CJ0011  62010CJ0420          0.709432   doc2vec_64_10   
8022   62001CJ0135  62001CJ0362          0.791363   doc2vec_64_10   
9020   62016CJ0384  62017CJ0145          0.598613  doc2vec_128_10   
9122   62013CJ0011  62010CJ0420          0.654122  doc2vec_128_10   
9284   62001CJ0135  62001CJ0362          0.703866  doc2vec_128_10   
1